In [1]:
using Latexify
using Distributions
using DataFrames
using CairoMakie

# Parameters
k_vals = [10, 100, 1000]  # Number of categories
count_vals = 10 .^ range(0, stop=15, length=100000)  # Total counts (c) on a logarithmic scale

# Initialize DataFrame to store results
results = DataFrame(k=Int[], case=String[], rel_interval=Float64[], c=Union{Int, Missing}[], mean=Union{Float64, Missing}[], lower=Union{Float64, Missing}[], upper=Union{Float64, Missing}[])

# Analyze each configuration
for k in k_vals
    cases = [("alpha_i = c / k + 1", c -> c / k + 1), ("alpha_i = c / 2 + 1", c -> c / 2 + 1)]
    for (label, alpha_func) in cases
        for threshold in [0.3, 0.1, 0.05]
            found = false
            for c in count_vals
                c = round(Int, c)
                alpha0 = k + c  # Total concentration
                alpha_i = alpha_func(c)  # Calculate alpha_i based on the case
                beta_i = alpha0 - alpha_i

                # Marginal Beta distribution
                beta_dist = Beta(alpha_i, beta_i)

                # Compute mean and 95% credible interval
                mean_val = alpha_i / alpha0
                lower = quantile(beta_dist, 0.025)
                upper = quantile(beta_dist, 0.975)
                relative_interval = (upper - lower) / mean_val

                if relative_interval < threshold
                    push!(results, (k, label, relative_interval, c, mean_val, lower, upper))
                    found = true
                    break
                end
            end
            if !found
                push!(results, (k, label, threshold, missing, missing, missing, missing))
            end
        end
    end
end



In [4]:

# Display the results
sorted = sort(results, [:case, :k, :rel_interval], rev=[true, false, true])[!, [:case,:k,:c, :mean, :lower, :upper, :rel_interval, ]]

# Add a column c / k
sorted[!, "c_div_k"] = [ismissing(row.c) ? missing : row.c / row.k for row in eachrow(sorted)]
println(sorted)

18×8 DataFrame
 Row │ case                 k      c        mean      lower        upper       rel_interval  c_div_k  
     │ String               Int64  Int64?   Float64?  Float64?     Float64?    Float64       Float64  
─────┼────────────────────────────────────────────────────────────────────────────────────────────────
   1 │ alpha_i = c / k + 1     10     1524  0.1       0.0854942    0.115493       0.299992    152.4
   2 │ alpha_i = c / k + 1     10    13819  0.1       0.0950552    0.105054       0.0999913  1381.9
   3 │ alpha_i = c / k + 1     10    55322  0.1       0.0975141    0.102513       0.049992   5532.2
   4 │ alpha_i = c / k + 1    100    16786  0.01      0.00855504   0.0115549      0.299982    167.86
   5 │ alpha_i = c / k + 1    100   152038  0.01      0.00950616   0.010506       0.0999886  1520.38
   6 │ alpha_i = c / k + 1    100   608426  0.01      0.00975154   0.0102515      0.0499976  6084.26
   7 │ alpha_i = c / k + 1   1000   169396  0.001     0.00085555   0.0011

In [12]:
# Function to round values and format rows with subtables and separate captions
function create_latex_table_with_subtables(data, case1_name, case2_name, columns)
    table = "\\begin{table}[h!]\n\\centering\n"
    
    # Subtable 1: Case 1
    table *= "\\begin{subtable}[t]{\\textwidth}\n\\centering\n"
    table *= "\\begin{tabular}{|r|r|r|r|r|r|}\n\\hline\n"
    table *= "\$k\$ & \$c\$ & Mean & Lower (\$2.5\\%\$) & Upper (\$97.5\\%\$)  & Relative Interval\\\\\n\\hline\n"
    prev_k = nothing
    for row in eachrow(data)
        if row.case == case1_name
            k = row.k
            c = row.c
            mean = isnothing(row.mean) ? "-" : round(row.mean, digits=3)
            rel_interval = round(row.rel_interval, digits=3)
            lower = round(row.lower, digits=5)
            upper = round(row.upper, digits=5)
            
            # Add an \hline whenever k changes
            if isnothing(prev_k) || k != prev_k
                table *= "\\hline\n"
            end
            
            table *= "\$$k\$ & \$$c\$ & \$$mean\$  & \$$lower\$ & \$$upper\$ & \$$rel_interval\$\\\\\n"
            prev_k = k
        end
    end
    table *= "\\hline\n\\end{tabular}\n"
    table *= "\\caption{Results for \$\\$case1_name\$}\n\\end{subtable}\n"
    
    # Subtable 2: Case 2
    table *= "\\begin{subtable}[t]{\\textwidth}\n\\centering\n"
    table *= "\\begin{tabular}{|r|r|r|r|r|r|}\n\\hline\n"
    table *= "\$k\$ & \$c\$ & Mean & Lower (\$2.5\\%\$) & Upper (\$97.5\\%\$)  & Relative Interval\\\\\n\\hline\n"
    prev_k = nothing
    for row in eachrow(data)
        if row.case == case2_name
            k = row.k
            c = row.c
            mean = isnothing(row.mean) ? "-" : round(row.mean, digits=3)
            rel_interval = round(row.rel_interval, digits=3)
            lower = round(row.lower, digits=5)
            upper = round(row.upper, digits=5)
            
            # Add an \hline whenever k changes
            if isnothing(prev_k) || k != prev_k
                table *= "\\hline\n"
            end
            
            table *= "\$$k\$ & \$$c\$ & \$$mean\$  & \$$lower\$ & \$$upper\$ & \$$rel_interval\$\\\\\n"
            prev_k = k
        end
    end
    table *= "\\hline\n\\end{tabular}\n"
    table *= "\\caption{Results for \$\\$case2_name\$}\n\\end{subtable}\n"
    
    # Close the main table structure
    table *= "\\caption{Results for Two Cases}\n\\end{table}"
    return table
end

# Create the table with two subtables
combined_table = create_latex_table_with_subtables(
    sorted,
    "alpha_i = c / k + 1",
    "alpha_i = c / 2 + 1",
    [:k, :c, :mean, :rel_interval, :lower, :upper]
)


println(combined_table)


\begin{table}[h!]
\centering
\begin{subtable}[t]{\textwidth}
\centering
\begin{tabular}{|r|r|r|r|r|r|}
\hline
$k$ & $c$ & Mean & Lower ($2.5\%$) & Upper ($97.5\%$)  & Relative Interval\\
\hline
\hline
$10$ & $1524$ & $0.1$  & $0.08549$ & $0.11549$ & $0.3$\\
$10$ & $13819$ & $0.1$  & $0.09506$ & $0.10505$ & $0.1$\\
$10$ & $55322$ & $0.1$  & $0.09751$ & $0.10251$ & $0.05$\\
\hline
$100$ & $16786$ & $0.01$  & $0.00856$ & $0.01155$ & $0.3$\\
$100$ & $152038$ & $0.01$  & $0.00951$ & $0.01051$ & $0.1$\\
$100$ & $608426$ & $0.01$  & $0.00975$ & $0.01025$ & $0.05$\\
\hline
$1000$ & $169396$ & $0.001$  & $0.00086$ & $0.00116$ & $0.3$\\
$1000$ & $1534306$ & $0.001$  & $0.00095$ & $0.00105$ & $0.1$\\
$1000$ & $6139993$ & $0.001$  & $0.00098$ & $0.00103$ & $0.05$\\
\hline
\end{tabular}
\caption{Results for $\alpha_i = c / k + 1$}
\end{subtable}
\begin{subtable}[t]{\textwidth}
\centering
\begin{tabular}{|r|r|r|r|r|r|}
\hline
$k$ & $c$ & Mean & Lower ($2.5\%$) & Upper ($97.5\%$)  & Relative Interval